<p>
<font size='5' face='Georgia, Arial'>IIC-2233 Apunte Programación Avanzada</font><br>
<font size='1'> Editado por Equipo Docente IIC2233 2019-1 al 2023-2.</font>
</p>

# Tabla de contenidos

1. [Conexiones entre múltiples ventanas en PyQt](#Conexiones-entre-múltiples-ventanas--en-PyQt)
    1. [¡Señales al rescate!](#¡Señales-al-rescate!)

## Conexiones entre múltiples ventanas  en PyQt

Al crear interfaces con múltiples ventanas, es natural y común querer comunicarlas y específicamente, cambiar entre una ventana y otra. Desde los ejemplos iniciales, uno puede verse tentado a utilizar la interfaz de `show` y `hide` para mostrar y ocultar ventanas. 

No sería raro inicialmente intentar algo como lo siguiente, dónde una ventana instancia otra e intenta abrirla.

Este código se encuentra en el archivo `5-ejemplo-conexion-entre-ventanas-ejemplo_1.py`.

```python
import sys
from PyQt6.QtWidgets import QApplication, QWidget, QPushButton


class Ventana(QWidget):

    def __init__(self, titulo, x, y):
        super().__init__()
        self.setWindowTitle(titulo)
        self.setGeometry(x, y, 200, 50)
        self.boton = QPushButton("Abrir otra ventana", self)
        self.boton.clicked.connect(self.abrir_otra_ventana)
        self.show()

    def abrir_otra_ventana(self):
        self.hide()  # Esconder la ventana actual
        otra_ventana = Ventana("Otra ventana", 300, 100)  # Crear otra
        otra_ventana.show()  # Mostrar nueva ventana
        
```

Si pruebas ejecutar lo anterior en tu computador, notarás un comportamiento innesperado: **no se muestra la segunda ventana**.

Intentemos un ángulo distinto, instanciemos la segunda ventana antes y la entregamos como un argumento al instanciar la ventana inicial.

Este código se encuentra en el archivo `5-ejemplo-conexion-entre-ventanas-ejemplo_2.py`.

```python
import sys
from PyQt6.QtWidgets import QApplication, QWidget, QPushButton


class Ventana(QWidget):

    def __init__(self, titulo, x, y, otra_ventana=None):
        super().__init__()
        self.otra_ventana = otra_ventana
        self.setWindowTitle(titulo)
        self.setGeometry(x, y, 200, 50)
        self.boton = QPushButton("Abrir otra ventana", self)
        self.boton.clicked.connect(self.abrir_otra_ventana)

    def abrir_otra_ventana(self):
        if self.otra_ventana is not None:
            self.hide()  # Esconder la ventana actual
            self.otra_ventana.show()  # Mostrar otra ventana


if __name__ == '__main__':
    app = QApplication([])
    # Segunda ventana se crea antes de forma independiente
    otra_ventana = Ventana("Otra ventana", 300, 100)
    # Ventana inicial recibe como argumento a otra_ventana
    ventana = Ventana("Inicial", 100, 100, otra_ventana)
    ventana.show()
    sys.exit(app.exec())

```

Este sí funciona si se intenta ejecutar. 🤔 ¿Qué ocurrió en el primero entonces? 🤔

El detalle, es que al instanciar un *widget* como una variable dentro de un método, como toda **variable local** del método, cuando se termine dicho método, Python **descarta** la variable.

```python
def abrir_otra_ventana(self):
    self.hide()
    otra_ventana = Ventana("Otra ventana", 300, 100)
    otra_ventana.show()
```

El método `abrir_otra_ventana` guarda en la variable `otra_ventana` el nuevo *widget* a mostrar. Pero, como `otra_ventana` es una variable local del método, cuando termine de ejecutarse, `otra_ventana` será descartada completamente, eliminando incluso el *widget* recién creado (y mostrado).

La diferencia en el segundo código, es que `otra_ventana` existe fuera de la definición del método, por lo que el hecho de que termine el método, no genera que se descarte la variable y objeto.

Siguiendo esa misma idea, entonces, almacenar la variable `otra_ventana` como un atributo de instancia debería también arreglar el problema. Y es cierto:

Este código se encuentra en el archivo `5-ejemplo-conexion-entre-ventanas-ejemplo_3.py`.

```python
import sys
from PyQt6.QtWidgets import QApplication, QWidget, QPushButton


class Ventana(QWidget):
    def __init__(self, titulo, x, y):
        super().__init__()
        self.setWindowTitle(titulo)
        self.setGeometry(x, y, 200, 50)
        self.boton = QPushButton("Abrir otra ventana", self)
        self.boton.clicked.connect(self.abrir_otra_ventana)

    def abrir_otra_ventana(self):
        self.hide()
        self.otra_ventana = Ventana("Otra ventana", 300, 100)
        self.otra_ventana.show()
```

🤔 ¿Por qué? 🤔

Porque si se almacena la segunda instancia de ventana como un atributo de la primera, al salir del método `abrir_otra_ventana` no se descarta al objeto ventana; este quedó referenciado en un nivel mayor al método: en la  instancia de la primera ventana.

Por lo tanto **siempre hay que guardar una referencia de nuestros objetos pyqt en memoria**, ya sea como atributo o dentro de una estructura de datos (lista por eemplo) y que dicha lista está como atributo.

#### ¿Es esa la mejor manera de modelar este comportamiento?

La solución anterior funciona. Y para casos pequeños, basta. El problema es en aplicaciones más grandes, donde múltiples ventanas pueden llamarse y hacerse aparecer. Es más, la modelación anterior muestra una dependencia entre las ventanas. En particular, la segunda solo existe bajo la existencia de la primera. Y esta modelación no aplica para todos los casos. 

Múltiples ventanas pueden ser independientes pero aún así puede gatillarse un evento que haga aparecer una desde otra. 

🤔 ¿Cómo puede modelarse esto? 🤔

### ¡Señales al rescate!

Como a casi todos nuestros problemas, señales son la solución. De forma similar a como se mostró en el cuaderno sobre *front-end* y *back-end*, señales ayudan a des-acoplar programas. De la misma forma, el uso de señales en este contexto es ventajoso ya que nos permite independizar ventanas y conectarlas mediante señales.

Este código se encuentra en el archivo `5-ejemplo-conexion-entre-ventanas-ejemplo_4.py`.

```python
import sys
from PyQt6.QtWidgets import QApplication, QWidget, QPushButton
from PyQt6.QtCore import pyqtSignal


class Ventana(QWidget):

    # Cada ventana se instancia con una señal para ser abierta
    senal_abrir_ventana = pyqtSignal()
    # Otra señal para avisar a una segunda ventana
    senal_abrir_otra_ventana = pyqtSignal()

    def __init__(self, titulo, x, y):
        super().__init__()
        # Definimos lo básico de la ventana
        self.setWindowTitle(titulo)
        self.setGeometry(x, y, 200, 50)

        # La señal que le permite a esta ventana abrirse,
        # se conecta a su propio show. Así, si alguien
        # emite la señal, esta ventana se mostrará
        self.senal_abrir_ventana.connect(self.show)

        # Creamos botón que se conecta a método self.abrir_otra_ventana
        self.boton = QPushButton("Abrir otra ventana", self)
        self.boton.clicked.connect(self.abrir_otra_ventana)

    def abrir_otra_ventana(self):
        self.hide()
        self.senal_abrir_otra_ventana.emit()


if __name__ == '__main__':
    app = QApplication([])

    # Instanciamos dos ventanas distintas
    # Cada una comienza con una señal propia que
    # le permite ser abierta por otra.
    ventana_1 = Ventana("Inicial", 100, 100)
    ventana_2 = Ventana("Alternativa", 500, 100)

    ventana_1.senal_abrir_otra_ventana.connect(ventana_2.show)
    ventana_2.senal_abrir_otra_ventana.connect(ventana_1.show)

    ventana_1.show()
    sys.exit(app.exec())
```

Así, cada instancia de `Ventana` es independiente. Es mediante señales que se comunican que una ventana debe aparecer luego de la otra, sin contener directamente una instancia dentro de otra instancia de `Ventana`.